<a href="https://colab.research.google.com/github/osori/korean-hate-comments/blob/master/good_comments_guardian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Korean HateSpeech Dataset
!wget https://github.com/kocohub/korean-hate-speech/raw/master/labeled/train.tsv
!wget https://github.com/kocohub/korean-hate-speech/raw/master/labeled/dev.tsv
!wget https://github.com/kocohub/korean-hate-speech/raw/master/test.no_label.tsv

--2020-09-11 20:00:57--  https://github.com/kocohub/korean-hate-speech/raw/master/labeled/train.tsv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kocohub/korean-hate-speech/master/labeled/train.tsv [following]
--2020-09-11 20:00:57--  https://raw.githubusercontent.com/kocohub/korean-hate-speech/master/labeled/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 913546 (892K) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>] 892.13K  --.-KB/s    in 0.07s   

2020-09-11 20:00:57 (11.9 MB/s) - ‘train.tsv’ saved [913546/913546]

--2020-09-11 20:00:57--  https://gi

In [2]:
!pip install soynlp
!pip install transformers

     |████████████████████████████████| 419kB 3.2MB/s 
     |████████████████████████████████| 890kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 15.6MB/s 
     |████████████████████████████████| 3.0MB 24.7MB/s 
     |████████████████████████████████| 890kB 28.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=aabfa91cf8f3276906a101b07984aa7392f981f66e07464e8013128afcb53370
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
# DistilKoBERT: Lightweight version of KoBERT
!git clone https://github.com/monologg/DistilKoBERT.git

Cloning into 'DistilKoBERT'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 544 (delta 95), reused 126 (delta 48), pack-reused 365
Receiving objects: 100% (544/544), 531.66 KiB | 5.02 MiB/s, done.
Resolving deltas: 100% (302/302), done.


In [4]:
import pandas as pd
import os

train_dataset = pd.read_csv('train.tsv', sep='\t')
val_dataset = pd.read_csv('dev.tsv', sep='\t')
test_dataset = pd.read_csv('test.no_label.tsv', sep='\t')

In [5]:
train_dataset.sample(5)

,comments,contain_gender_bias,bias,hate
3484,세경씨 그렇게 안봤는데 진짜 무서운사람이네 세경씨,False,none,none
1827,느무 재미 없다.,False,none,none
3982,아니 왜 범죄자가 기어나오냐고 자꾸,False,none,hate
641,광희야 방탄 대신 군대 10년만 더 갔다오자 애국하는거야,False,none,offensive
2094,돈없어봐라ㅋㅋ ㅈㄹ들떠세요,False,none,offensive


In [6]:
train_dataset['comments'].apply(len).mean()

38.710739614994935

In [7]:
from soynlp.normalizer import *

# remove emoticons and repeated characters like "ㅋㅋㅋ"
def clean(text):
  text = text.map(lambda x: emoticon_normalize(x, num_repeats=3))
  text = text.map(lambda x: repeat_normalize(x, num_repeats=2))
  return text

# map 1 if offensive or hate
#     0 if none
def map_hate(hate_value):
  mapping = {
      'none': 0,
      'offensive': 1,
      'hate': 1
  }
  hate_value = hate_value.map(lambda x: mapping.get(x))
  return hate_value

train_dataset['comments'] = clean(train_dataset['comments'])
val_dataset['comments'] = clean(val_dataset['comments'])
test_dataset['comments'] = clean(test_dataset['comments'])

train_dataset['hate'] = map_hate(train_dataset['hate'])
val_dataset['hate'] = map_hate(val_dataset['hate'])

In [8]:
# Load DistilKoBERT tokenizer
import transformers
from DistilKoBERT.tokenization_kobert import KoBertTokenizer

distilbert_model = transformers.DistilBertModel.from_pretrained('monologg/distilkobert')
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

In [9]:
tokenizer.tokenize("근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데")

['▁근',
 '데',
 '▁',
 '만원',
 '이',
 '하는',
 '▁현금',
 '결제',
 '만',
 '▁',
 '하라고',
 '▁써',
 '놓',
 '은',
 '집',
 '▁우리나라',
 '에',
 '▁엄',
 '청',
 '▁많은',
 '데']

In [10]:
import logging
logging.basicConfig(level=logging.ERROR)

from tqdm import tqdm
import numpy as np

# convert to BERT-understandable data
def convert_data(data_df):
    global tokenizer

    SEQ_LEN = 64
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True, truncation=True)
       
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        segment = [0]*SEQ_LEN
 
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        targets.append(data_df[LABEL_COLUMN][i])
 
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)
 
    return [tokens, masks, segments], targets

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y
 
SEQ_LEN = 64
BATCH_SIZE = 32
DATA_COLUMN = "comments"
LABEL_COLUMN = "hate"
 
train_x, train_y = load_data(train_dataset)


  0%|          | 0/7896 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 7896/7896 [00:02<00:00, 3334.78it/s]


In [11]:
test_x, test_y = load_data(val_dataset)

  0%|          | 0/471 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 471/471 [00:00<00:00, 3227.74it/s]


In [15]:
from keras.optimizers import *
import tensorflow as tf
import tensorflow_addons as tfa

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*8, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08)

print("All devices: ", tf.config.list_logical_devices('TPU'))

def create_hate_bert():
  model = transformers.TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

  bert_outputs = bert_outputs[1]

  hate_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
  hate_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(hate_drop)
  hate_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], hate_first)

  return hate_model

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.110.193.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.110.193.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


In [17]:
with strategy.scope():
  hate_model = create_hate_bert()
  hate_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
  hate_model.fit(train_x, train_y, epochs=6, shuffle=True, batch_size=64, validation_data=(test_x, test_y))
  # hate_model.save('hate_model.h5') 

print(hate_model.summary())
loss, acc = hate_model.evaluate(test_x, test_y, verbose=2)
print('Model accuracy: {:5.2f}%'.format(100*acc))

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/6
  2/124 [..............................] - ETA: 45:24 - loss: 0.7116 - accuracy: 0.4531  WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0096s vs `on_train_batch_end` time: 0.0913s). Check your callbacks.


124/124 [==============================] - ETA: 0s - loss: 0.6875 - accuracy: 0.5408WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0036s vs `on_test_batch_end` time: 0.0267s). Check your callbacks.


124/124 [==============================] - 103s 834ms/step - loss: 0.6875 - accuracy: 0.5408 - val_loss: 0.6603 - val_accuracy: 0.5987
Epoch 2/6
124/124 [==============================] - 13s 101ms/step - loss: 0.6780 - accuracy: 0.5640 - val_loss: 0.6688 - val_accuracy: 0.5541
Epoch 3/6
124/124 [==============================] - 12s 100ms/step - loss: 0.6385 - accuracy: 0.6320 - val_loss: 0.5542 - val_accuracy: 0.7091
Epoch 4/6
124/124 [==============================] - 12s 99ms/step - loss: 0.5694 - accuracy: 0.7077 - val_loss: 0.5223 - val_accuracy: 0.7495
Epoch 5/6
124/124 [==============================] - 12s 100ms/step - loss: 0.5193 - accuracy: 0.7392 - val_loss: 0.4741 - val_accuracy: 0.7792
Epoch 6/6
124/124 [==============================] - 12s 100ms/step - loss: 0.4762 - accuracy: 0.7748 - val_loss: 0.5972 - val_accuracy: 0.7176
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)             

15/15 - 4s - loss: 0.5972 - accuracy: 0.7176
Model accuracy: 71.76%


In [18]:
hate_model.save_weights('/content/hate_bert_model.h5')

In [23]:
def sentence_convert_data(data):
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True, truncation=True)
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def predict_comment(sentence):
    data_x = sentence_convert_data(sentence)
    predict = hate_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer:
      print("*", sentence, "(악플 확률 : %.2f)" % predict_value)
    else:
      print(sentence, "(악플 확률 : %.2f)" % predict_value)

In [24]:
# test set prediction
for comment in test_dataset['comments'][:100]:
  predict_comment(comment)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


ㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ (악플 확률 : 0.44)
둘다 넘 좋다~행복하세요 (악플 확률 : 0.09)
* 근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데 (악플 확률 : 0.70)
원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요 (악플 확률 : 0.08)
장현승 얘도 참 이젠 짠하다... (악플 확률 : 0.35)
신선하게 웃긴ㅋㅋㅋ시 동엽신~~!! 장소연님은 진짜 조선족인가 착각할정도로 말투가 리얼하네요 (악플 확률 : 0.14)
* 누군데 얘네? (악플 확률 : 0.64)
* 하자 인생들 모아다가 방송에 내보내고, 덜 하자가 교정해서 장사 풀리게 해주는 감동 스토리 백하자의 골목식당. 호텔 말고 그냥 하자 거리를 하나 열어서 거기다 하자 인생들 교화소를 만들지... 왜 저러고 살까... (악플 확률 : 0.56)
진짜 라디오 스타 노래한거 보세요 홍진영은비비지도 못함 (악플 확률 : 0.35)
* 쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 기어나올 생각말고 하던대로 그냥 조용히 살어라! 잠재적 살인마. (악플 확률 : 0.88)
* 질투 ㅈㄴ 하네 뚱땡이년들 수지보다는 훨씬낫다 (악플 확률 : 0.92)
안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고??눈 감았다가 뜨는 동영상 올려보시죠...정작 안검하수 있는 분들은 수술해도 부작용 때문에.. 눈꺼플 잘 못감거든요.. (악플 확률 : 0.21)
그건 불가능해. 그냥 은퇴해서 평범하게 살아라. 언제가 될지는 모르지만, 시기가 문제일뿐... 망가지는건 분명하다. 연예계에서 사는거보다 일반인으로 사는게 행복할거다. (악플 확률 : 0.32)
약사는 역시 김재환이지 (악플 확률 : 0.10)
* 발연기 현빈 국밥장인 팬들만 물고 빠는 ㅋㅋㅋ (악플 확률 : 0.77)
* 골목살리고 지가하는 체인점 다입점해서. 때돈벌고. 피디 술사주고. 지가 음식점